In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

domo_instance = os.environ["DOMO_INSTANCE"]
domo_username = os.environ["DOMO_USERNAME"]
domo_password = os.environ["DOMO_PASSWORD"]

In [ ]:
from selenium import webdriver


def driversetup(is_headless: bool = True) -> webdriver:
    options = webdriver.ChromeOptions()
    # run Selenium in headless mode

    if is_headless:
        options.add_argument("--headless")

    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=options)

    return driver

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time


def wait_and_return(
    driver: webdriver,
    element_id: str,
    el_type=By.ID,
    min_sleep_time=15,
    return_soup: bool = False,
):
    """
    Wait for the element to be present and return it.
    """

    try:
        res = WebDriverWait(driver, timeout=15, poll_frequency=1).until(
            EC.presence_of_element_located((el_type, element_id)))

        if return_soup:
            return BeautifulSoup(res.get_attribute("innerHTML"))
        return res
    
    except Exception as e:
        print(e)
        print(
            f"Timeout Exception: did not load within {min_sleep_time} seconds.")
    

In [ ]:
# Waiting for the page to load
def authenticate_driver(driver, domo_instance, domo_username, domo_password):
    
    url = f"https://{domo_instance}.domo.com/auth/index?redirectUrl=%2Fapi%2Fdomoweb%2Fauth%2Fcustomerportal%3FredirectPath%3D%2Fs%2F"

    driver.get(url)

    button = wait_and_return(driver, element_id="sign-in", el_type=By.CLASS_NAME, min_sleep_time=4)

    form_username = wait_and_return(driver, "username", el_type=By.NAME)

    form_password = wait_and_return(driver, "password", el_type=By.NAME)

    # Sending input
    form_username.clear()
    form_username.send_keys(domo_username)
    form_password.clear()
    form_password.send_keys(domo_password)
    button.click()

    return driver


driver = driversetup(is_headless=False)

driver = authenticate_driver(
    driver,
    domo_instance=domo_instance,
    domo_password=domo_password,
    domo_username=domo_username,
)

In [ ]:
import time

def get_all_tickets_soup(driver):
    url = f"https://domo-support.domo.com/s/"

    driver.get(url)

    # browser element
    table_elem = wait_and_return(
        driver, "slds-grid", el_type=By.CLASS_NAME, return_soup=False, min_sleep_time=15
    )

    #try get all table elements
    len_table = 0
    while len_table == 0:
        print(f"💤 sleeping {len_table} 💤")
        time.sleep(2)
        table_soup = BeautifulSoup(table_elem.get_attribute("innerHTML"))

        len_table = len(table_soup.find_all("td"))

    # try scroll
    new_len_table = len_table
    prev_len_table = 0
    while new_len_table != prev_len_table:
        scroll_bar = wait_and_return(
            driver, "uiScroller", el_type=By.CLASS_NAME,  min_sleep_time=15)
        scroll_bar.send_keys(Keys.END)

        time.sleep(2)
        print(f"💤📜 sleeping on the scroll {len_table}, {new_len_table} 💤📜")

        table_soup = BeautifulSoup(table_elem.get_attribute("innerHTML"))
        prev_len_table = new_len_table
        new_len_table = len(table_soup.find_all("td"))
    
    print(f"🚀 done retrieving table {new_len_table} 🚀")

    return table_soup.find("table")


ticket_soup = get_all_tickets_soup(driver)


💤 sleeping 0 💤
💤📜 sleeping on the scroll 400, 400 💤📜
💤📜 sleeping on the scroll 400, 720 💤📜
🚀 done retrieving table 720 🚀


In [ ]:
import pandas as pd


def generate_ticket_df(soup, return_raw: bool = False, base_url="https://domo-support.domo.com"):
    df_with_links = pd.read_html(str(soup), extract_links="body")[0]
    df = pd.read_html(str(soup))[0]

    rename_cols = {
        col: col.replace("Column Actions", "")
        .replace("Sorted", "")
        .replace("Sort", "")
        .replace("Ascending", "")
        .replace("Descending", "")
        .strip()
        for col in df.columns
    }

    df.rename(columns=rename_cols, inplace=True)
    df_with_links.rename(columns=rename_cols, inplace=True)

    if return_raw:
        return (df, df_with_links)


    df_with_links = pd.DataFrame(
        df_with_links["Case Number"].tolist(),
        index=df_with_links.index,
        columns=["Case Number", "URL"],
    )

    df["Case Number"] = df["Case Number"].astype(int)
    df_with_links["Case Number"] = df_with_links["Case Number"].astype(int)

    df = pd.merge(df, df_with_links, on="Case Number", how="inner")

    df['URL'] = base_url + df['URL']

    print(f"🚀 {len(df.index)} tickets retrieved")

    return df

df = generate_ticket_df(ticket_soup, return_raw= False)
df.columns

🚀 90 tickets retrieved


Index(['Item Number', 'Case Number', 'Support Category', 'Subject', 'Status',
       'Requester', 'Domain Name', 'Date/Time Opened', 'Action', 'URL'],
      dtype='object')

In [ ]:
def get_article(authenticated_driver, url):
    authenticated_driver.get(url)

    description_soup = wait_and_return(
        authenticated_driver,
        "test-id__record-layout-container",
        el_type=By.CLASS_NAME,
        return_soup=True,
    )

    casecomments_soup = wait_and_return(
        authenticated_driver,
        "caseCommentsContainer",
        el_type=By.CLASS_NAME,
        return_soup=True,
        min_sleep_time=5,
    )

    casecomments_ls = []
    for li in casecomments_soup.find("ul").find_all("li"):
        comment_hdr = li.find(class_="date").text
        comment_body = li.find(class_="commentBody").text

        casecomments_ls.append({"date": comment_hdr, "body": comment_body})

    return {"description": description_soup.text, "case_comments": casecomments_ls}


TEST_URL = "https://domo-support.domo.com/s/case/5005w00002Boo4oAAB/please-update-all-the-feature-switches"

article = get_article(driver, TEST_URL)

In [ ]:
import domolibrary.utils.convert as dmcv
import domolibrary.client.DomoAuth as dmda
import math
import numbers

from dataclasses import dataclass, field


@dataclass
class AppDbRecord:
    content: dict = field(repr=False)
    content_id: str
    document_id: str = None
    datastore_id: str = None
    collection_id: str = None
    auth: dmda.DomoAuth = field(default=None, repr=False)

    def __eq__(self, other):
        return self.content_id == other.content_id and \
            self.datastore_id == other.datastore_id and \
            self.collection_id == other.collection_id

    @classmethod
    def from_json(cls, doc_obj, id_col, auth=None, datastore_id=None, collection_id=None):

        res = {}

        for key in doc_obj.keys():

            if not doc_obj[key] or (isinstance(doc_obj[key], numbers.Number) and math.isnan(doc_obj[key])):
                continue

            key_clean = key.replace('/', ' ').replace('Descending', '')
            key_clean = dmcv.convert_snake_to_pascal(key_clean)

            res.update({key_clean: doc_obj[key]})

        return cls(content=res,
                   content_id=doc_obj.get(id_col),
                   auth=auth,
                   datastore_id=datastore_id,
                   collection_id=collection_id)

    @classmethod
    def from_appdb(cls, document, id_col: str = None, auth: dmda.DomoAuth = None):

        content = document.get('content')

        return cls(content=content,
                   content_id=content.get(
                       id_col) if id_col else content.get('content_id'),
                   document_id=document.get('id'),
                   datastore_id=document.get('datastoreId'),
                   collection_id=document.get('collectionId'),
                   auth=auth)

    def to_document(self):
        content = self.content
        content.update({'content_id': self.content_id})

        return {
            'content': self.content
        }

    def delete(self, auth: dmda.DomoAuth = None):
        auth = auth or self.auth

        url = f'https://{auth.domo_instance}.domo.com/api/datastores/v1/{self.datastore_id}/collections/{self.collection_id}/documents/{self.document_id}'

        return request(method='DELETE', url=url, headers=auth.auth_header)

    @classmethod
    def create(cls, auth: dmda.DomoAuth, datastore_id, collection_id, doc_obj, id_col):

        app_rec = cls.from_json(
            doc_obj, id_col, datastore_id=datastore_id, collection_id=collection_id, auth=auth)

        url = f'https://{auth.domo_instance}.domo.com/api/datastores/v1/{datastore_id}/collections/{collection_id}/documents/'

        res = request(method='POST', url=url, headers=auth.auth_header,
                       json=app_rec.to_document())
        
        return res

    
    def update(self):
        url = f'https://{auth.domo_instance}.domo.com/api/datastores/v1/{self.datastore_id}/collections/{self.collection_id}/documents/{self.document_id}'

        return request(method='PUT', url=url, headers=auth.auth_header,
                       json=self.to_document())


In [ ]:
# pip install domolibrary


In [ ]:
def get_documents(auth, datastore_id, collection_id, id_col :str = None , return_raw: bool = False):
    url = f'https://{domo_instance}.domo.com/api/datastores/v1/{datastore_id}/collections/{collection_id}/documents/'

    res = request(method='GET', url=url, headers=auth.auth_header)

    document_ls = res.json()

    if return_raw:
        return document_ls

    return [AppDbRecord.from_appdb(document, id_col=id_col, auth = auth ) for document in document_ls]



In [ ]:
import domolibrary.client.DomoAuth as dmda
from requests import request

auth = dmda.DomoFullAuth(domo_username=domo_username,
                         domo_instance=domo_instance, domo_password=domo_password)

await auth.get_auth_token()


datastore_id = 'de349e92-186b-40fd-8b1d-c1ac120bde64'
collection_id = '6fa667b4-aa4d-465d-8ebc-540e94c0f0a1'

domo_record_ls = get_documents(auth, datastore_id=datastore_id,
              collection_id=collection_id, 
              id_col = 'caseNumber',
              return_raw=False)

domo_record_ls

[AppDbRecord(content_id=5845813, document_id='33d89549-b0c8-445e-8018-45a2981cc4d3', datastore_id='de349e92-186b-40fd-8b1d-c1ac120bde64', collection_id='6fa667b4-aa4d-465d-8ebc-540e94c0f0a1')]

In [ ]:
# test delete record
# !for domo_record in domo_record_ls:
#     !domo_record.delete()

domo_record_ls = get_documents(auth, datastore_id=datastore_id,
                               collection_id=collection_id,
                               id_col='caseNumber',
                               return_raw=False)

domo_record_ls


[]

In [ ]:
def upsert_document(auth, datastore_id, collection_id, doc_obj, id_col):
    new_doc = AppDbRecord.from_json(doc_obj=doc_obj, id_col=id_col, auth = auth, datastore_id= datastore_id, collection_id= collection_id)

    exist_documents = get_documents(auth, datastore_id, collection_id)

    match_doc = next((e_doc for e_doc in exist_documents if e_doc == new_doc), None)

    if match_doc:
        match_doc.content = new_doc.content
        return match_doc.update()
    
    return AppDbRecord.create(doc_obj=doc_obj, id_col=id_col, auth = auth, datastore_id= datastore_id, collection_id= collection_id)
    

In [ ]:
def get_processed_tickets(datastore_id, collection_id, auth):
    all_docs= get_documents(auth, datastore_id=datastore_id,
                                   collection_id=collection_id,
                                   return_raw=False)
    
    return [ doc.content_id for doc in all_docs]

auth = dmda.DomoFullAuth(domo_username=domo_username,
                         domo_instance=domo_instance, domo_password=domo_password)

await auth.get_auth_token()

datastore_id = 'de349e92-186b-40fd-8b1d-c1ac120bde64'
collection_id = '6fa667b4-aa4d-465d-8ebc-540e94c0f0a1'

processed_tickets = get_processed_tickets(datastore_id = datastore_id, collection_id = collection_id, auth = auth)
processed_tickets


[5845813]

In [ ]:
def process_tickets(row, processed_tickets, datastore_id, collection_id, auth):

    if row['Case Number'] in processed_tickets:
        return
    
    article_dict = get_article(driver, row['URL'])
    article_dict.update(row.to_dict())


    new_record =  AppDbRecord.create(auth=auth, datastore_id=datastore_id,
                       collection_id=collection_id, doc_obj=article_dict, id_col='Case Number')

    processed_tickets.append(row['Case Number'])

    return new_record


[process_tickets(row, processed_tickets, datastore_id= datastore_id, collection_id= collection_id, auth = auth) for index, row in df.iterrows()]

[None,
 None,
 None,
 None,
 None,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200